In [1]:
import torch
from transformers import BertTokenizer
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:

########################### amz_train #####################################
# data_df = pd.read_csv("HMC_data/text/train_40k.csv")
# label_df = pd.read_csv("HMC_data/text/amz_labels_training_40k.csv")

########################### amz_validate #####################################
# data_df = pd.read_csv("HMC_data/text/val_10k.csv")
# label_df = pd.read_csv("HMC_data/text/amz_labels_validation_10k.csv")
# print(data_df.size, label_df.size)
############################# enron_trainvalid ###################################
data_df = pd.read_csv("HMC_data/text/enron_labeled_text.csv")
label_df = pd.read_csv("HMC_data/text/enron_labels.csv")
data_df = data_df.assign(Label=label_df["Label"])
############################# enron_test ###################################

FileNotFoundError: [Errno 2] No such file or directory: 'HMC_data/text/enron_labeled_text.csv'

In [111]:
####### Tokenization ######
# Filter out data that are 
## 1. not labeled
## 2. longer than 512 tokens as contraint by BERT tokenizer

# Extract text content
# text = data_df['Text'] ### for amz
text = data_df['content'] ### for enron

# Clean text data
text = text.dropna()
text = text.str.lower()
text = text.str.replace('[^a-zA-Z]', ' ')

# Tokenize text data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', device=device)

# Define a function to tokenize the text
def tokenize_text(text):
    max_length = 512
    return tokenizer.encode(text, add_special_tokens=True, max_length=max_length)

# Apply the tokenize_text function to the column containing text
data_df['tokenized_text'] = text.apply(tokenize_text)
data_df = data_df.dropna(subset=['tokenized_text'])
# data_df['tokenized_text'].fillna("", inplace=True)

# Apply padding to make sure all sequence have the same length
max_length = 512
tokenized_text_padded = np.vstack((np.array(x + [0]*(max_length - len(x))) for x in data_df['tokenized_text'].values))

/var/folders/sb/w27b1569221gwdwr8_btsbpm0000gr/T/ipykernel_48854/1037384979.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace('[^a-zA-Z]', ' ')
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/var/folders/sb/w27b1569221gwdwr8_btsbpm0000gr/T/ipykernel_48854/1037384979.py:30: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  tokenized_text_padded = np.vstack((np.array(x + [0]*(max_length - len(x))) for x in data_df['tokenized_text

In [112]:
############# amz #########################################
# unique_vals = label_df["Label"].unique()
############## enron ##################################################
unique_vals = "1,1/1,1/1/1,1/1/2,1/1/3,1/1/4,1/1/5,1/1/6,1/1/7,1/1/8,1/1/9,1/1/10,1/1/11,1/1/12,1/1/13,1/2,1/3,1/4,1/5,1/6,1/7,1/8,2,2/1,2/2,2/3,2/4,2/5,2/6,2/7,2/8,2/9,2/10,2/11,2/12,2/13,4,4/1,4/2,4/3,4/4,4/5,4/6,4/7,4/8,4/9,4/10,4/11,4/12,4/13,4/14,4/15,4/16,4/17,4/18,4/19".split(",")
len(unique_vals)

56

In [113]:
### write arff
## 1. corpus as attributes - no we dont need this, just attrX
## 2. all the unique labels
## 3. encodings per sample

# Define the function to convert CSV to ARFF
def tensor_to_arff(tensor, labels, unique_labels, output_file, relation_name):
    ## input parameters:
    ### tensor - input tensor data without label
    ### labels - list of labels in tensor
    ### unique_labels - list of unique labels in tensor

    # Write the ARFF file
    with open(output_file, 'w') as arfffile:
        # Write the relation name
        arfffile.write('@RELATION ' + relation_name + '\n\n')

        # Write the attributes
        N, T = tensor.shape
        for i in range(1, T+1):
            arfffile.write('@ATTRIBUTE a'+ str(i) + '  numeric\n')

        # Write the unique labels
        arfffile.write('@ATTRIBUTE class  hierarchical '+ ','.join([str(label) for label in unique_labels]) + '\n')

        # Write the data
        arfffile.write('\n@DATA\n')
        for j in range(0, N):
            # Data + Data label
            arfffile.write(','.join([str(val) for val in tensor[j]]) + ',' + str(labels[j]) + '\n')

In [114]:
# Example usage:
################################ amz ################################
# tensor_to_arff(tokenized_text_padded, label_df["Labels"].tolist(), unique_vals, 'amz_review_others.arff', 'amz_review_others')
# tensor_to_arff(tokenized_text_padded, label_df["Label"].tolist(), unique_vals, 'amz_review_others_validate.arff', 'amz_review_others_validate')
#################################### enron ############################################################
tensor_to_arff(tokenized_text_padded, data_df["Label"].tolist(), unique_vals, 'enron_text_others_train.arff', 'enron_text_others_train')